# 심층 신경망

## 교제 데이터 사용

In [1]:
import pandas as pd
import numpy as np

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()

In [3]:
fish_target = fish['Species'].to_numpy()

In [4]:
from sklearn.preprocessing import LabelEncoder
# 타겟 데이터 인코딩
label_encoder = LabelEncoder()
fish_target = label_encoder.fit_transform(fish_target)

In [5]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


## 샘플링 편향 문제 해결

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

In [7]:
print(train_input.shape, test_input.shape)

(119, 5) (40, 5)


In [8]:
print(train_target.shape, test_target.shape)

(119,) (40,)


In [9]:
print(test_target)

[2 5 3 6 2 0 5 4 2 3 0 6 0 1 0 0 2 2 2 0 5 0 0 0 0 2 2 6 5 5 3 2 2 3 0 2 4
 4 1 2]


## 데이터 전처리

In [10]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [11]:
print(ss.mean_, ss.scale_)

[393.27226891  28.49663866  31.2697479    8.86248403   4.44828655] [355.27097574  10.67118899  11.5129708    4.04941829   1.64047206]


## 심층 신경망 만들기

In [12]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
from tensorflow import keras
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(7, activation='relu', input_shape=(5,)),
])

/opt/miniconda3/envs/hb/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 100 == 0:  # 100 단위 에포크에서만 출력
            print(f"Epoch {epoch}, Loss: {logs['loss']}, Accuracy: {logs['accuracy']}, Val Loss: {logs['val_loss']}, Val Accuracy: {logs['val_accuracy']}")


In [16]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # 검증 손실을 모니터링
    patience=10,         # 10 에포크 동안 개선 없으면 조기 종료
    restore_best_weights=True  # 가장 좋은 모델로 복원
)


In [17]:
import keras_tuner as kt

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        activation='relu', input_shape=(train_scaled.shape[1],)))
    model.add(keras.layers.Dense(
        units=hp.Int('units', min_value=32, max_value=256, step=32),
        activation='relu'))
    model.add(keras.layers.Dense(len(pd.unique(fish_target)), activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='intro_to_kt')

tuner.search(train_scaled, train_target, epochs=10, validation_split=0.2, callbacks=[early_stopping])


Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [18]:
# 하이퍼파라미터 튜닝 후 최적의 모델 선택
best_model = tuner.get_best_models(num_models=1)[0]

# 최적의 하이퍼파라미터를 사용하여 모델 재학습
history = best_model.fit(
    train_scaled, train_target,
    epochs=1000,
    validation_split=0.2,
    callbacks=[CustomCallback(), early_stopping])

# 모델 평가
test_loss, test_accuracy = best_model.evaluate(test_scaled, test_target)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


Epoch 1/1000


/opt/miniconda3/envs/hb/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.8125 - loss: 0.3912Epoch 0, Loss: 0.9085057377815247, Accuracy: 0.7263157963752747, Val Loss: 1.2901545763015747, Val Accuracy: 0.6666666865348816
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.7616 - loss: 0.7239 - val_accuracy: 0.6667 - val_loss: 1.2902
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7567 - loss: 0.7781 - val_accuracy: 0.7917 - val_loss: 0.6048
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7748 - loss: 0.4916 - val_accuracy: 0.8333 - val_loss: 0.6412
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8404 - loss: 0.4579 - val_accuracy: 0.7917 - val_loss: 0.5869
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8023 - loss: 0.4032 - val_accuracy: 0.7500 - val_loss: 0.5795
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8273 - loss: 0.3642 - val_accuracy: 0.8333 - val_loss: 0.5219
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accu